In [1]:
import re
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from datetime import datetime, timedelta

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1702579954635_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#prefix list

#dt: datetime
#str: string

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
dt_proc_brazilian = datetime.now() - timedelta(hours=3)
str_proc_brazilian_datetime = dt_proc_brazilian.strftime("%Y-%m-%dT%H:%M:%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
spark = SparkSession.builder.appName('raw_customers').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
str_landzone_file_path = "ecommerce/olist_customers_dataset_20231214170632.csv"
str_file_name = str_landzone_file_path.split("/")[-1]
print(str_file_name)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

olist_customers_dataset_20231214170632.csv

In [21]:
str_file_extract_pattern = re.compile(r'(\d{14})')
str_file_extract_datetime = str_file_extract_pattern.search(str_file_name).group(1)

dt_file_extract = datetime.strptime(str_file_extract_datetime, "%Y%m%d%H%M%S")
str_file_extract_datetime = dt_file_extract = dt_file_extract.strftime("%Y-%m-%dT%H:%M:%S")
print(str_file_extract_datetime)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2023-12-14T17:06:32

In [22]:
str_bucket_landzone = "ecommerce-project-landzone"
str_bucket_raw = "ecommerce-project-raw"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
str_raw_file_name = "ecommerce/olist_customers_dataset"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
str_s3_landzone_file_path = f's3://{str_bucket_landzone}/{str_landzone_file_path}'
print(str_s3_landzone_file_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://ecommerce-project-landzone/ecommerce/olist_customers_dataset_20231214170632.csv

In [26]:
landzone_customers = spark.read.format(
    "com.databricks.spark.csv").option(
    "header","true").option(
    "encoding","UTF-8").option(
    "inferSchema","false").option(
    "delimiter",",").load(
    str_s3_landzone_file_path)

landzone_customers.createOrReplaceTempView("landzone_customers")
landzone_customers.cache()
qtd=landzone_customers.count()
print('rows from landzone file: ', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rows from landzone file:  99441

In [27]:
for col in landzone_customers.columns:
    print(col)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

customer_id
customer_unique_id
customer_zip_code_prefix
customer_city
customer_state

In [28]:
raw_customers = spark.sql(
    f"""
        SELECT
            --default lake metadata
            int(date_format(current_date, 'yyyyMM')) as ref,
            int(date_format(current_date, 'yyyyMM')) as ref_partition,
            "{dt_file_extract}" as dt_file_extraction,
            "{dt_file_extract}" as dt_file_extraction_partition,
            "{str_proc_brazilian_datetime}" as dt_proc,
            "{str_proc_brazilian_datetime}" as dt_file_extraction_partition,
            --file fields
            customer_id,
            customer_unique_id,
            customer_zip_code_prefix,
            customer_city,
            customer_state
        FROM landzone_customers
    """
)

raw_customers.createOrReplaceTempView('raw_customers')
raw_customers.cache()
raw_customers.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

99441

In [29]:
raw_customers.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- dt_file_extraction: string (nullable = false)
 |-- dt_file_extraction_partition: string (nullable = false)
 |-- dt_proc: string (nullable = false)
 |-- dt_file_extraction_partition: string (nullable = false)
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

In [30]:
raw_customers.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+-------------------+----------------------------+-------------------+----------------------------+--------------------+--------------------+------------------------+--------------------+--------------+
|   ref|ref_partition| dt_file_extraction|dt_file_extraction_partition|            dt_proc|dt_file_extraction_partition|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+------+-------------+-------------------+----------------------------+-------------------+----------------------------+--------------------+--------------------+------------------------+--------------------+--------------+
|202312|       202312|2023-12-14T17:06:32|         2023-12-14T17:06:32|2023-12-14T18:09:12|         2023-12-14T18:09:12|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|202312|       202312|2023-12-14T17:06:32|         2023-12-14T17:06:32|2023-12-14T18:09:12|         2023

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'str_s3_raw_file_path' is not defined
Traceback (most recent call last):
NameError: name 'str_s3_raw_file_path' is not defined

